In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('global_terrorism.csv', encoding = 'latin1')

C:\Users\joshy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#raised dtype warning complaining about Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types

mixed_indexes = [4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121]

for index in mixed_indexes:
    print(list(df.columns)[index])

approxdate
resolution
attacktype2_txt
attacktype3_txt
targsubtype3_txt
gsubname2
gname3
gsubname3
claimmode2_txt
claimmode3_txt
weaptype3_txt
weapsubtype3_txt
weaptype4_txt
weapsubtype4_txt
divert
kidhijcountry
ransomnote


In [2]:
#restrict to columns I am interested in
#interested in: iyear, imonth, region, crit1, crit2, crit3,
#success, suicide, attacktype1, attacktype2, attacktype3, targtype1, targtype2, targtype3
#weaptype1, ...
#individual (unaffiliated), claimed
#nkill, nwound, property, propextent (probably collapse these two- propextent is 1-4 if property is yes)

#hierarchy of attacktype is interesting- maybe should pivot 9 different columns

for i in range(len(list(df.columns))):
    print(i, list(df.columns)[i])

0 eventid
1 iyear
2 imonth
3 iday
4 approxdate
5 extended
6 resolution
7 country
8 country_txt
9 region
10 region_txt
11 provstate
12 city
13 latitude
14 longitude
15 specificity
16 vicinity
17 location
18 summary
19 crit1
20 crit2
21 crit3
22 doubtterr
23 alternative
24 alternative_txt
25 multiple
26 success
27 suicide
28 attacktype1
29 attacktype1_txt
30 attacktype2
31 attacktype2_txt
32 attacktype3
33 attacktype3_txt
34 targtype1
35 targtype1_txt
36 targsubtype1
37 targsubtype1_txt
38 corp1
39 target1
40 natlty1
41 natlty1_txt
42 targtype2
43 targtype2_txt
44 targsubtype2
45 targsubtype2_txt
46 corp2
47 target2
48 natlty2
49 natlty2_txt
50 targtype3
51 targtype3_txt
52 targsubtype3
53 targsubtype3_txt
54 corp3
55 target3
56 natlty3
57 natlty3_txt
58 gname
59 gsubname
60 gname2
61 gsubname2
62 gname3
63 gsubname3
64 motive
65 guncertain1
66 guncertain2
67 guncertain3
68 individual
69 nperps
70 nperpcap
71 claimed
72 claimmode
73 claimmode_txt
74 claim2
75 claimmode2
76 claimmode2_txt

In [3]:
col_indexes =  [1,2,9,19,20,21,26,27,28,30,32,34,42,50,68,81,85,89,93,98,101,104,105]

terrorism_df = pd.read_csv('global_terrorism.csv', encoding = 'latin1', usecols = col_indexes, 
                           na_values = {'attacktype1': 9, 'weaptype1': 13, 'targtype1': 20, 'property': -9, 'propextent':4})
#na_values from codebook
sum(terrorism_df['success'])
#terrorism_df.head()

152701

In [34]:
terrorism_df.info()
#attacktype2, attacktype3, weaptype2, weaptype3, weaptype4 have lots of missing values, but that's ok if i'm going to pivot them
#nkill and nwound have some
#propextent has a lot of missing values, but that might be fine since I can fill them easily based on property

#there are 152701 successful attacks out of 170350 total

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170350 entries, 0 to 170349
Data columns (total 23 columns):
iyear          170350 non-null int64
imonth         170350 non-null int64
region         170350 non-null int64
crit1          170350 non-null int64
crit2          170350 non-null int64
crit3          170350 non-null int64
success        170350 non-null int64
suicide        170350 non-null int64
attacktype1    163925 non-null float64
attacktype2    5630 non-null float64
attacktype3    374 non-null float64
targtype1      165477 non-null float64
targtype2      10018 non-null float64
targtype3      1034 non-null float64
individual     170350 non-null int64
weaptype1      156498 non-null float64
weaptype2      11843 non-null float64
weaptype3      1660 non-null float64
weaptype4      74 non-null float64
nkill          160668 non-null float64
nwound         155025 non-null float64
property       150771 non-null float64
propextent     41479 non-null float64
dtypes: float64(14), int64

In [7]:
terrorism_df[pd.isnull(terrorism_df['nkill'])]

#from codebook: Where there is evidence of fatalities, but a figure is not reported or it is too vague to be of use, 
#this field remains blank

,iyear,imonth,region,crit1,crit2,crit3,success,suicide,attacktype1,attacktype2,...,targtype3,individual,weaptype1,weaptype2,weaptype3,weaptype4,nkill,nwound,property,propextent
3,1970,1,8,1,1,1,1,0,3.0,NaN,...,NaN,0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,1970,1,4,1,1,1,1,0,7.0,NaN,...,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
15,1970,1,9,1,1,1,1,0,3.0,NaN,...,NaN,0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
34,1970,1,9,1,1,1,1,0,7.0,NaN,...,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
95,1970,2,10,1,1,1,1,0,2.0,NaN,...,NaN,0,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
176,1970,4,3,1,1,0,1,0,2.0,NaN,...,NaN,0,5.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0
241,1970,4,9,1,1,1,1,0,7.0,NaN,...,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
277,1970,5,8,1,1,1,1,0,3.0,NaN,...,NaN,0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
288,1970,5,8,1,1,1,1,0,7.0,NaN,...,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
290,1970,5,8,1,1,1,1,0,2.0,NaN,...,NaN,0,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [8]:
terrorism_df.describe()

,iyear,imonth,region,crit1,crit2,crit3,success,suicide,attacktype1,attacktype2,...,targtype3,individual,weaptype1,weaptype2,weaptype3,weaptype4,nkill,nwound,property,propextent
count,170350.000000,170350.000000,170350.000000,170350.000000,170350.000000,170350.000000,170350.000000,170350.000000,163925.000000,5630.000000,...,1034.000000,170350.000000,156498.000000,11843.000000,1660.000000,74.000000,160668.000000,155025.000000,150771.000000,41479.000000
mean,2001.709997,6.474365,7.091441,0.988441,0.992985,0.879818,0.896396,0.033871,2.993674,3.672647,...,9.882012,0.002483,5.846925,6.741366,6.869880,6.243243,2.387246,3.200239,0.597058,2.977917
std,13.144146,3.392364,2.949206,0.106888,0.083461,0.325175,0.304747,0.180899,1.535013,2.245530,...,5.803961,0.049769,0.939205,2.196341,2.157719,1.497128,11.327709,34.647365,0.490491,0.147938
min,1970.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,1.000000,2.000000,5.000000,0.000000,0.000000,0.000000,1.000000
25%,1990.000000,4.000000,5.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,2.000000,...,3.000000,0.000000,5.000000,5.000000,5.000000,5.000000,0.000000,0.000000,0.000000,3.000000
50%,2007.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,2.000000,...,14.000000,0.000000,6.000000,6.000000,6.000000,6.000000,0.000000,0.000000,1.000000,3.000000
75%,2014.000000,9.000000,10.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,6.000000,...,14.000000,0.000000,6.000000,8.000000,8.000000,6.000000,2.000000,2.000000,1.000000,3.000000
max,2016.000000,12.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,9.000000,...,22.000000,1.000000,12.000000,13.000000,13.000000,12.000000,1500.000000,7366.000000,1.000000,3.000000


In [19]:
#clearly the distribution is right-skewed, but the highest values don't seem to be outliers per se, so I don't think 
#I need to remove any

nwound = np.sort(terrorism_df[~pd.isnull(terrorism_df['nwound'])]['nwound'])[::-1]
print(nwound[:1000])

[ 7366.  7365.  5500.  4000.  1500.  1272.  1001.   817.   800.   751.
   750.   750.   727.   700.   671.   650.   600.   552.   542.   500.
   450.   450.   450.   450.   433.   400.   400.   400.   400.   391.
   386.   370.   360.   360.   351.   347.   347.   340.   319.   301.
   300.   300.   300.   300.   300.   300.   300.   300.   296.   295.
   286.   285.   276.   270.   260.   257.   250.   250.   250.   250.
   250.   250.   250.   250.   246.   245.   238.   236.   235.   235.
   233.   230.   230.   227.   227.   224.   224.   220.   220.   220.
   217.   217.   217.   216.   215.   215.   211.   207.   201.   201.
   201.   200.   200.   200.   200.   200.   200.   200.   200.   200.
   200.   200.   200.   200.   200.   200.   200.   200.   200.   200.
   200.   200.   200.   200.   200.   197.   194.   192.   188.   184.
   183.   183.   182.   180.   178.   178.   177.   176.   175.   173.
   173.   171.   171.   170.   170.   170.   170.   170.   169.   168.
   168

In [11]:
#I think this is a similar situation

#now I'd like to "pivot" the attacktype and targettype columns, since some attacks have multiple types
#and the order in which they are listed is not that meaningful

sum(terrorism_df['success'])

152701

In [43]:
'''
attacktype1    163925 non-null float64
targtype1      165477 non-null float64
weaptype1      156498 non-null float64
nkill          160668 non-null float64
nwound         155025 non-null float64
property       150771 non-null float64
propextent
'''


In [9]:
#make this less repetitive
cleaned_df = terrorism_df[~pd.isnull(terrorism_df['attacktype1'])]
cleaned_df = cleaned_df[~pd.isnull(cleaned_df['targtype1'])]
cleaned_df = cleaned_df[~pd.isnull(cleaned_df['weaptype1'])]
cleaned_df = cleaned_df[~pd.isnull(cleaned_df['nkill'])]
cleaned_df = cleaned_df[~pd.isnull(cleaned_df['nwound'])]
cleaned_df = cleaned_df[~pd.isnull(cleaned_df['property'])]

cleaned_df = cleaned_df.reset_index(drop=True)

In [10]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122376 entries, 0 to 122375
Data columns (total 23 columns):
iyear          122376 non-null int64
imonth         122376 non-null int64
region         122376 non-null int64
crit1          122376 non-null int64
crit2          122376 non-null int64
crit3          122376 non-null int64
success        122376 non-null int64
suicide        122376 non-null int64
attacktype1    122376 non-null float64
attacktype2    3533 non-null float64
attacktype3    214 non-null float64
targtype1      122376 non-null float64
targtype2      6685 non-null float64
targtype3      703 non-null float64
individual     122376 non-null int64
weaptype1      122376 non-null float64
weaptype2      8547 non-null float64
weaptype3      1127 non-null float64
weaptype4      63 non-null float64
nkill          122376 non-null float64
nwound         122376 non-null float64
property       122376 non-null float64
propextent     37549 non-null float64
dtypes: float64(14), int64(9)

In [16]:
len(cleaned_df) - sum(cleaned_df['success'])

#about 10% unsuccessful

#reasonable proportion of points left.

10033

In [17]:
multiple_targets = cleaned_df[~pd.isnull(cleaned_df['targtype2'])]

multiple_targets[['targtype1', 'targtype2', 'targtype3']].head()

#weirdly, does not seem to be hierarchical

,targtype1,targtype2,targtype3
102,14.0,3.0,NaN
122,1.0,8.0,NaN
156,3.0,1.0,NaN
186,8.0,14.0,NaN
297,14.0,1.0,NaN


In [29]:
#the difference in mean and max of nkill was pretty striking, so I investigated the top numbers killed of the dataframe
nkill = np.sort(cleaned_df[~pd.isnull(cleaned_df['nkill'])]['nkill'])[::-1]
print(nkill[:825])

#clearly the distribution is right-skewed, but the highest values don't seem to be outliers per se, so I'm not sure
#I need to remove any

[ 1383.  1382.   670.   422.   344.   300.   287.   284.   259.   250.
   250.   230.   224.   224.   200.   189.   188.   180.   170.   168.
   158.   152.   150.   145.   141.   130.   127.   122.   120.   119.
   112.   110.   108.   108.   106.   103.   103.   102.   101.   101.
   101.   101.   101.   100.   100.   100.   100.   100.   100.    98.
    98.    97.    97.    96.    93.    91.    90.    89.    88.    88.
    87.    86.    83.    82.    82.    81.    80.    80.    80.    79.
    79.    77.    77.    76.    75.    75.    75.    74.    74.    74.
    74.    73.    73.    72.    72.    72.    72.    71.    70.    70.
    70.    70.    69.    69.    68.    68.    68.    67.    67.    66.
    66.    66.    65.    65.    65.    65.    64.    64.    64.    64.
    64.    63.    63.    63.    63.    63.    63.    62.    62.    62.
    62.    62.    62.    62.    61.    61.    61.    60.    60.    60.
    60.    60.    60.    60.    60.    59.    59.    59.    58.    58.
    58

In [30]:
nwound = np.sort(cleaned_df[~pd.isnull(cleaned_df['nwound'])]['nwound'])[::-1]
print(nwound[:825])

[ 7366.  7365.  4000.  1500.   817.   751.   750.   750.   727.   650.
   600.   552.   500.   450.   450.   450.   450.   400.   400.   400.
   360.   360.   351.   340.   301.   300.   300.   300.   285.   276.
   270.   260.   250.   250.   250.   246.   227.   220.   220.   217.
   211.   201.   200.   200.   200.   200.   200.   200.   200.   200.
   200.   200.   200.   182.   178.   178.   175.   173.   171.   170.
   170.   168.   167.   163.   162.   161.   161.   160.   160.   160.
   160.   160.   159.   158.   157.   154.   152.   151.   150.   150.
   150.   150.   150.   150.   150.   150.   150.   150.   150.   150.
   150.   150.   150.   149.   148.   148.   147.   145.   143.   141.
   140.   140.   140.   140.   137.   136.   136.   134.   132.   131.
   131.   130.   130.   130.   130.   130.   130.   128.   127.   125.
   125.   125.   125.   123.   123.   122.   122.   122.   121.   120.
   120.   120.   119.   119.   119.   115.   115.   114.   113.   110.
   110

In [55]:
cleaned_df = pd.read_csv('cleaned.csv')

In [57]:
region_dict = {1: 'North America', 2: 'Central America', 3: 'South America', 4: 'East Asia', 5: 'SE Asia', 6: 'South Asia', 
7: 'Central Asia', 8: 'West Europe',9: 'East Europe',10: 'ME and North Africa', 
11: 'Sub-Saharan Africa', 12: 'Oceania'}

#leave one out
for i in range(1,12):
    cleaned_df[region_dict[i]] = (cleaned_df['region'] == i).astype(int)
    
print(cleaned_df[['region','North America','South America', 'ME and North Africa']])

        region  North America  South America  ME and North Africa
0            1              1              0                    0
1            3              0              1                    0
2            1              1              0                    0
3            1              1              0                    0
4            1              1              0                    0
5            1              1              0                    0
6            1              1              0                    0
7            8              0              0                    0
8            1              1              0                    0
9            1              1              0                    0
10           1              1              0                    0
11           1              1              0                    0
12           1              1              0                    0
13           3              0              1                    0
14        

In [10]:
#cleaned_df.loc['weaptype1' == 12].describe()
#12 = other

cleaned_df.loc[cleaned_df['weaptype1'] == 12].describe()

,Unnamed: 0,iyear,imonth,region,crit1,crit2,crit3,success,suicide,attacktype1,...,targtype3,individual,weaptype1,weaptype2,weaptype3,weaptype4,nkill,nwound,property,propextent
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.0,97.000000,97.000000,97.0,97.000000,...,0.0,97.000000,97.0,4.000000,0.0,0.0,97.000000,97.000000,97.000000,17.000000
mean,68845.587629,2003.381443,6.597938,6.793814,0.989691,1.0,0.979381,0.752577,0.0,6.711340,...,NaN,0.010309,12.0,8.750000,NaN,NaN,1.134021,0.793814,0.422680,2.882353
std,33542.038948,11.479550,3.193939,3.523546,0.101535,0.0,0.142842,0.433756,0.0,1.701462,...,NaN,0.101535,0.0,3.304038,NaN,NaN,3.858510,3.138929,0.496552,0.332106
min,1946.000000,1976.000000,1.000000,1.000000,0.000000,1.0,0.000000,0.000000,0.0,1.000000,...,NaN,0.000000,12.0,5.000000,NaN,NaN,0.000000,0.000000,0.000000,2.000000
25%,41517.000000,1992.000000,4.000000,4.000000,1.000000,1.0,1.000000,1.000000,0.0,6.000000,...,NaN,0.000000,12.0,7.250000,NaN,NaN,0.000000,0.000000,0.000000,3.000000
50%,63809.000000,2007.000000,6.000000,8.000000,1.000000,1.0,1.000000,1.000000,0.0,7.000000,...,NaN,0.000000,12.0,8.500000,NaN,NaN,0.000000,0.000000,0.000000,3.000000
75%,99522.000000,2014.000000,10.000000,10.000000,1.000000,1.0,1.000000,1.000000,0.0,8.000000,...,NaN,0.000000,12.0,10.000000,NaN,NaN,1.000000,0.000000,1.000000,3.000000
max,122104.000000,2016.000000,12.000000,11.000000,1.000000,1.0,1.000000,1.000000,0.0,8.000000,...,NaN,1.000000,12.0,13.000000,NaN,NaN,30.000000,25.000000,1.000000,3.000000


In [47]:
weapon_dict = {1:'Biological',2:'Chemical',3:'Radiological',
            4:'Nuclear',5:'Firearms',6:'Explosives',
            7:'Fake Weapon', 8:'Incendiary', 9:'Melee',
            10:'Vehicle',11:'Sabotage Equipment', 12:'Other'}

#leave out 'other'
for i in range(1,12):
    cleaned_df[weapon_dict[i]] = ((cleaned_df['weaptype1'] == i) | (cleaned_df['weaptype2'] == i) 
                                   | (cleaned_df['weaptype3'] == i) | (cleaned_df['weaptype4'] == i)).astype(int)
    
multiple_types = cleaned_df.loc[pd.notnull(cleaned_df['weaptype3'])]
    
print(multiple_types[['weaptype1', 'weaptype2', 'weaptype3','Firearms', 'Explosives','Melee']])

        weaptype1  weaptype2  weaptype3  Firearms  Explosives  Melee
415           6.0        5.0       13.0         1           1      0
543           5.0        5.0        5.0         1           0      0
1111          5.0        9.0        9.0         1           0      1
1113          5.0        9.0        9.0         1           0      1
1142          5.0        6.0        8.0         1           1      0
1151          9.0        9.0        9.0         0           0      1
1228          5.0        5.0        5.0         1           0      0
2866          6.0        6.0        6.0         0           1      0
3178          5.0        6.0        6.0         1           1      0
3581          5.0        5.0        6.0         1           1      0
3763          5.0        5.0        5.0         1           0      0
4124          5.0        6.0        5.0         1           1      0
4409          5.0        5.0        5.0         1           0      0
4505          5.0        6.0      

In [46]:
type_dict = {1:'Assassination', 2:'Armed Assault',
            3:'Bombing', 4:'Hijacking', 5:'Hostage (Barricade)',
             6:'Kidnapping', 7:'Infrastructure', 8:'Unarmed Assault'}

#leave out the last, again
for i in range(1,8):
    cleaned_df[type_dict[i]] = ((cleaned_df['attacktype1'] == i) | (cleaned_df['attacktype2'] == i)
                                | (cleaned_df['attacktype3'] == i)).astype(int)
    
multiple_a_types = cleaned_df.loc[pd.notnull(cleaned_df['attacktype3'])]
    
print(multiple_a_types[['attacktype1', 'attacktype2', 'attacktype3','Bombing','Infrastructure']])

        attacktype1  attacktype2  attacktype3  Bombing  Infrastructure
980             5.0          7.0          2.0        0               1
45696           3.0          2.0          7.0        1               1
52868           2.0          7.0          3.0        1               1
53240           3.0          2.0          7.0        1               1
53558           2.0          3.0          6.0        1               0
53874           2.0          6.0          7.0        0               1
54660           2.0          6.0          4.0        0               0
54691           2.0          3.0          6.0        1               0
55067           2.0          6.0          7.0        0               1
55638           2.0          3.0          1.0        1               0
55694           3.0          2.0          7.0        1               1
56041           2.0          7.0          7.0        0               1
56381           6.0          7.0          2.0        0               1
56451 

In [45]:
target_dict = {1:'Business', 2:'Government', 3:'Police',4:'Military',5:'Abortion',6:'Aviation',7:'Diplomatic',8:'Education',9:'FoodWater',10:'Media',
              11:'Maritime', 12:'NGO',13:'Other',14:'Private',15:'Religious',16:'Telecommunication',
              17:'Terrorists', 18:'Tourists', 19:'Transportation',21:'Utilities',22:'Violent Parties'}

target_codes = [i for i in range(1,23) if i != 13 and i != 20]

for i in target_codes:
    cleaned_df[target_dict[i]] = ((cleaned_df['targtype1'] == i) | (cleaned_df['targtype2'] == i)
                                | (cleaned_df['targtype3'] == i)).astype(int)
    
multiple_t_types = cleaned_df.loc[pd.notnull(cleaned_df['targtype3'])]
    
print(multiple_t_types[['targtype1', 'targtype2', 'targtype3','Government','Private']])

        targtype1  targtype2  targtype3  Government  Private
960           3.0       14.0        1.0           0        1
2087          7.0        2.0        2.0           1        0
3063          1.0        1.0        1.0           0        0
3064          1.0        1.0        1.0           0        0
3065          1.0        1.0        1.0           0        0
4477          3.0        2.0        3.0           1        0
6268          6.0        6.0        6.0           0        0
8529          7.0        7.0        7.0           0        0
8785          2.0        2.0        2.0           1        0
8853          4.0        4.0        4.0           0        0
9058          2.0        7.0       15.0           1        0
9539          2.0        2.0       14.0           1        1
9691          1.0       14.0       14.0           0        1
9742          2.0        2.0       14.0           1        1
10577         7.0        7.0        7.0           0        0
10879        22.0       

In [53]:
cleaned_df.to_csv('cleaned.csv')